In [1]:
import numpy as np

from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector

In [2]:
# Optimized parameters

parameters = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]

In [3]:

def construct_qcc_circuit(entanglers: list, truncation=None):
    '''This function defines the QCC ansatz circuit for VQE. Here we construct exponential blocks using
    entanglers from QMF state as a proof of principle demonstration.
    
    Args:
        entanglers: list storing Pauli words for construction of qcc_circuit.
        backend: statevector, qasm simulator or a real backend.
        truncation: a threshold number to truncate the blocks. Default: None.
    Returns:
        qcc_circuit
    '''
    if truncation != None:
        if len(entanglers) > truncation:
            num_blocks = truncation
        else:
            num_blocks = len(entanglers)
    else:
        num_blocks = len(entanglers)
    
    # p = ParameterVector('p', num_blocks)
    p = [0.0944527, 0.04799566, -0.0590973, -0.05908328, 0.04114604, 0.02695483, 0.02604318, 0.03485649]
    
    num_qubits = len(entanglers[0])
    qcc_circuit = QuantumCircuit(num_qubits)
    for i in range(num_blocks):
        circuit = QuantumCircuit(num_qubits)
        key = entanglers[i]
        coupler_map = []
        # We first construct coupler_map according to the key.
        for j in range(num_qubits):
            if key[num_qubits-1-j] != 'I':
                coupler_map.append(j)
                
        # Then we construct the circuit.
        if len(coupler_map) == 1:
            # there is no CNOT gate.
            c = coupler_map[0]
            if key[num_qubits-1-c] == 'X':
                circuit.h(c)
                circuit.rz(p[i], c)
                circuit.h(c)
            elif key[num_qubits-1-c] == 'Y':
                circuit.rx(-np.pi/2, c)
                circuit.rz(p[i], c)
                circuit.rx(np.pi/2, c)
                
            qcc_circuit += circuit
        else:
            # Here we would need CNOT gate.
            for j in coupler_map:
                if key[num_qubits-1-j] == 'X':
                    circuit.h(j)
                elif key[num_qubits-1-j] == 'Y':
                    circuit.rx(-np.pi/2, j)
                    
            for j in range(len(coupler_map) - 1):
                circuit.cx(coupler_map[j], coupler_map[j+1])
                
            param_gate = QuantumCircuit(num_qubits)
            param_gate.rz(p[i], coupler_map[-1])
            
            #qcc_circuit += circuit + param_gate + circuit.inverse()
            qcc_circuit.compose(circuit, inplace=True)
            qcc_circuit.compose(param_gate, inplace=True)
            qcc_circuit.compose(circuit.inverse(), inplace=True)
    
    # Would the optimization level setting to 3 represent the best Qiksit optimization?
    # trans_circuit = transpile(qcc_circuit, backend=backend, optimization_level=3)
    
    return qcc_circuit

def hf_circ(num_qubits):
    hf_circuit = QuantumCircuit(num_qubits)
    hf_circuit.x(0)
    hf_circuit.x(3)
        
    entanglers = ['XXXXXY', 'XXXIYI', 'IXIXXY', 'IXIIYI', 'IXXIXY', 'XXIXYI', 'IIIXIY', 'XIYIII']

    parameterized_circuit = hf_circuit.compose(construct_qcc_circuit(entanglers))
    
    return parameterized_circuit

In [4]:
hf_circuit = hf_circ(6)

hf_circuit.draw()

┌───┐┌──────────┐                                                        »
q_0: ┤ X ├┤ Rx(-π/2) ├──■─────────────────────────────────────────────────────»
     ├───┤└──────────┘┌─┴─┐                                                   »
q_1: ┤ H ├────────────┤ X ├──■────────────────────────────────────────────────»
     ├───┤            └───┘┌─┴─┐                                              »
q_2: ┤ H ├─────────────────┤ X ├──■────────────────────────────────────────■──»
     ├───┤   ┌───┐         └───┘┌─┴─┐                                    ┌─┴─┐»
q_3: ┤ X ├───┤ H ├──────────────┤ X ├──■──────────────────────────────■──┤ X ├»
     ├───┤   └───┘              └───┘┌─┴─┐                          ┌─┴─┐├───┤»
q_4: ┤ H ├───────────────────────────┤ X ├──■────────────────────■──┤ X ├┤ H ├»
     ├───┤                           └───┘┌─┴─┐┌──────────────┐┌─┴─┐├───┤├───┤»
q_5: ┤ H ├────────────────────────────────┤ X ├┤ Rz(0.094453) ├┤ X ├┤ H ├┤ H ├»
     └───┘                                └───┘└──────────────┘└───┘└───┘└───┘»
«               ┌─────────┐┌──────────┐                                    »
«q_0: ───────■──┤ Rx(π/2) ├┤ Rx(-π/2) ├────────────────────────────────────»
«          ┌─┴─┐└──┬───┬──┘├──────────┤                                    »
«q_1: ──■──┤ X ├───┤ H ├───┤ Rx(-π/2) ├──■─────────────────────────────────»
«     ┌─┴─┐├───┤   ├───┤   └──────────┘  │                                 »
«q_2: ┤ X ├┤ H ├───┤ H ├─────────────────┼─────────────────────────────────»
«     ├───┤├───┤   └───┘               ┌─┴─┐                               »
«q_3: ┤ H ├┤ H ├───────────────────────┤ X ├──■────────────────────────────»
«     ├───┤└───┘                       └───┘┌─┴─┐                          »
«q_4: ┤ H ├─────────────────────────────────┤ X ├──■────────────────────■──»
«     └───┘                                 └───┘┌─┴─┐┌──────────────┐┌─┴─┐»
«q_5: ───────────────────────────────────────────┤ X ├┤ Rz(0.047996) ├┤ X ├»
«                                                └───┘└──────────────┘└───┘»
«                                                                              »
«q_0: ────────────────────────────■─────────────────────────────────────────■──»
«               ┌─────────┐┌───┐┌─┴─┐                                     ┌─┴─┐»
«q_1: ───────■──┤ Rx(π/2) ├┤ H ├┤ X ├──■───────────────────────────────■──┤ X ├»
«            │  └─────────┘└───┘└───┘┌─┴─┐                           ┌─┴─┐├───┤»
«q_2: ───────┼───────────────────────┤ X ├──■─────────────────────■──┤ X ├┤ H ├»
«          ┌─┴─┐   ┌───┐   ┌───┐     └───┘  │                     │  └───┘└───┘»
«q_3: ──■──┤ X ├───┤ H ├───┤ H ├────────────┼─────────────────────┼────────────»
«     ┌─┴─┐├───┤   ├───┤   └───┘          ┌─┴─┐┌───────────────┐┌─┴─┐┌───┐┌───┐»
«q_4: ┤ X ├┤ H ├───┤ H ├──────────────────┤ X ├┤ Rz(-0.059097) ├┤ X ├┤ H ├┤ H ├»
«     ├───┤├───┤   └───┘                  └───┘└───────────────┘└───┘└───┘└───┘»
«q_5: ┤ H ├┤ H ├───────────────────────────────────────────────────────────────»
«     └───┘└───┘                                                               »
«     ┌─────────┐┌──────────┐                                                »
«q_0: ┤ Rx(π/2) ├┤ Rx(-π/2) ├─────────────────────────────────────────────■──»
«     └──┬───┬──┘├──────────┤                           ┌─────────┐┌───┐┌─┴─┐»
«q_1: ───┤ H ├───┤ Rx(-π/2) ├──■─────────────────────■──┤ Rx(π/2) ├┤ H ├┤ X ├»
«        ├───┤   └──────────┘  │                     │  └─────────┘└───┘└───┘»
«q_2: ───┤ H ├─────────────────┼─────────────────────┼───────────────────────»
«        └───┘                 │                     │                       »
«q_3: ─────────────────────────┼─────────────────────┼───────────────────────»
«                            ┌─┴─┐┌───────────────┐┌─┴─┐   ┌───┐   ┌───┐     »
«q_4: ───────────────────────┤ X ├┤ Rz(-0.059083) ├┤ X ├───┤ H ├───┤ H ├─────»
«                            └───┘└───────────────┘└───┘   └───┘   └───┘     »
«q_5: ────────────────────────────────────